# MDP and gym

## Evaluation considerations
- We take into account the correctness of the solutions but also their generality and quality of the code
- Comment and discuss on the results of all your exercises (in a cell immediately after the results). You may also state the difficulties encountered, lessons learned and your understanding of the problem and solution
- Clean-up your code before submission, do not leave unnecessary code attempts, or if you deem it important, leave them in a way that it is easily understood and with comments/discussion
- We also value the originality of the solutions, don't hesitate in performing unrequested additional tasks in relation to the exercises


**NOTE**

Do not try to reproduce exactly the results in this notebook. RL training is very noisy and performances of learned policies can vary a lot, try running your trains several times.



In [2]:
!pip install gymnasium
!pip install gymnasium[accept-rom-license,toy_text]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 KB 9.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 22.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached libtorrent-2.0.7-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (8.6 MB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.5.5-py3-none-any.whl size=441098 sha256=552a89ee9f5c1549fe6a4edfb5e71e1e378878010b948fdede700b6ea3ab5637
  Stored in directory: /root/.cache/pip/wheels/c3/86/6f/e96885ff274388b9f0636418a2926f46f076cd7e891670321d
Successfully built AutoROM.accept-rom-license


In [3]:
import gymnasium as gym
from IPython.display import clear_output, HTML, display
import matplotlib.pyplot as plt
%matplotlib notebook

In [4]:
#@title Wrapper for recording an environment into a video

from __future__ import annotations

from copy import deepcopy
from typing import Any, SupportsFloat

from gymnasium.core import ActType, ObsType, RenderFrame, WrapperActType, WrapperObsType
from gymnasium.error import DependencyNotInstalled

class RecordVideo(gym.Wrapper):
    """Adapted from https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/experimental/wrappers/rendering.py#L87
    """

    def __init__(self, env):
        """Initialize a :class:`HumanRendering` instance.
        Args:
            env: The environment that is being wrapped
        """
        super().__init__(env)
        assert env.render_mode in [
            "rgb_array",
            "rgb_array_list",
        ], f"Expected env.render_mode to be one of 'rgb_array' or 'rgb_array_list' but got '{env.render_mode}'"

        if "render_fps" not in env.metadata:
            env.metadata["render_fps"] = 24

        assert (
            "render_fps" in env.metadata
        ), "The base environment must specify 'render_fps' to be used with the HumanRendering wrapper"

        if "human" not in self.metadata["render_modes"]:
            self.metadata = deepcopy(self.env.metadata)
            self.metadata["render_modes"].append("human")

        self.artists = []
        self.figure = None

    @property
    def render_mode(self):
        """Always returns ``'human'``."""
        return "human"

    def step(
        self, action: WrapperActType
    ) -> tuple[WrapperObsType, SupportsFloat, bool, bool, dict]:
        """Perform a step in the base environment and render a frame to the screen."""
        result = super().step(action)
        self._render_frame()
        return result

    def reset(
        self, *, seed: int | None = None, options: dict[str, Any] | None = None
    ) -> tuple[WrapperObsType, dict[str, Any]]:
        """Reset the base environment and render a frame to the screen."""
        result = super().reset(seed=seed, options=options)
        self._render_frame()
        return result

    def video(self):
        """This method renders all frames collected up to now."""
        if self.figure is not None:
            from IPython.display import HTML
            import matplotlib.animation

            animation = matplotlib.animation.ArtistAnimation(self.figure, self.artists, 
                                                             interval=1000//self.metadata["render_fps"],
                                                             blit=True,
                                                             repeat=True,
                                                             repeat_delay=2000)
            return HTML(animation.to_html5_video())

        return None

    def _render_frame(self):
        """Fetch the last frame from the base environment and render it to the screen."""
        try:
            import matplotlib.animation
            import numpy as np
        except ImportError:
            raise DependencyNotInstalled(
                "matplotlib is not installed, run `pip install matplotlib`"
            )
        if self.env.render_mode == "rgb_array_list":
            rgb_arrays = self.env.render()
        elif self.env.render_mode == "rgb_array":
            rgb_arrays = [self.env.render()]
        else:
            raise Exception(
                f"Wrapped environment must have mode 'rgb_array' or 'rgb_array_list', actual render mode: {self.env.render_mode}"
            )

        assert isinstance(rgb_arrays, list)

        for rgb_array in rgb_arrays:
            assert isinstance(rgb_array, np.ndarray)

        if self.figure is None:
            self.figure = plt.figure()
            plt.axis('off')
        
        self.artists.append([plt.imshow(rgb_array) for rgb_array in rgb_arrays])

    def close(self):
        """Close the rendering window."""
        result = self.video()
        super().close()

        return result

In [5]:
import numpy as np
np.set_printoptions(linewidth=100)

Let's render soe steps to show how to use the `RecordVideo` class.

In [6]:
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
env.reset()                    

for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    done = terminated or truncated

    if done:
        env.reset()

display(env.video())

<IPython.core.display.Javascript object>

This is only needed when we want to render. In many situations of this lab we will perform many steps without the need to render, we will avoid recording a video, since doing so requires a lot of RAM (to save all the RGB frames) and we may run out of memory.

To avoid this do not wrap the environment on a `RecordVideo` class and set keyword argument `render_mode=None` when making the environment.

In [7]:
env = gym.make("FrozenLake-v1", render_mode=None)
env.reset()                    

for _ in range(100):
    observation, reward, terminated, truncated, info = env.step(env.action_space.sample())

    done = terminated or truncated

    if done:
        env.reset()

In this lab we will be estimating value functions. Value functions of an MDP are conditioned on the policy used.

A policy is the probability distribution over actions. We will restrict ourselves to discrete action spaces.

We will start by using a uniform policy that chooses with equal chances between all actions.

## Exercise : A uniform policy

Create a `policy_uniform` function that returns the probability of each possible action given an environment and state.

We will also create a `sample_multinomial` function that facilitates sampling an action from the policy given the probability of each possible action.

**Hint** you may access the number of discrete actions with `env.action_space.n`.

In [8]:
class UniformPolicy(object):
    def __init__(self, action_space):
        assert isinstance(action_space, gym.spaces.discrete.Discrete), "Can only create uniform policies for Discrete action spaces"
        self.n_actions = action_space.n
        self.training = True

    def train(self):
        self.training = True

    def eval(self):
        self.training = False

    def probability(self, state, action):
        ### BEGIN SOLUTION
        action_prob=1/self.n_actions
        ### END SOLUTION
        return action_prob

    def sample(self, state):
        ### BEGIN SOLUTION
        # action=env.action_space.sample()
        action=[0,1,2,3]
        action=np.random.choice(action)
        ### END SOLUTION

        return action

# Let's instantiate a uniform policy
env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
uniform_policy = UniformPolicy(env.action_space)

# And sample 20 actions from state==0
actions = [uniform_policy.sample(0) for i in range(20)]
print(actions)

[3, 3, 3, 0, 2, 3, 3, 0, 1, 0, 2, 1, 3, 1, 1, 1, 0, 3, 0, 0]


**Comments:**


for this unifrom policy,probability of each action is equal and it's value equal to 1/4 , and the action is one of available four actions (such as moving up, down, left, or right), then we print the random choosing action 20 times.


Gym follows a formalism of MDPs that differs slightly (is more general) than the one we have seen in class. The differences are:
- In gym, transitions from a particular state and action can lead to the same destination state with different rewards (in the course all transitions reaching the same state obtain the same reward, except for transitions departing from a terminal state, whose rewards can be ignored)
- In gym, transitions to the same state can be both terminal and non-terminal (in the course it's the states that are terminal)

However we have chosen the environment `FrozenLake` which behaves as the MDPs described in the course, thus:
- All transitions to a given state have the same reward (unless the departure and destination state is the same terminal state)
- All transitions to a terminal state are terminal

In bym `env.env.P` contains the information about the process dynamics, rewards and terminal states. The information is encoded in the following way:

`env.env.P[state][action]` returns a list of tuples containing all the possible outcomes and their probability in the form `(prob, next_state, reward, terminal)` where `state` is the current state, `action` is the action taken, `probability` is the probability of the transition, `next_state` is the destination state of the transition, `reward` is the reward obtained for the transition and `terminal` is a boolean indicating if the transition ends the episode.

## Exercise : MDP from gym environment

Using this information compute the matrices $\mathcal{P}_{ss'}$, $\mathcal{R}_s$ and `terminal_states` of your MDP. Terminal is a boolean vector containing `True` for terminal states and `False` otherwise.

We will use the uniform policy that we created before, that assigns equal probability to all actions.

**Hint** all the transitions from a terminal state must be ignored.

**Optionally** while computing the matrices verify with `assert` that the conditions that we described above are fulfilled.

{state: {action: [(probability, next_state, reward, is_next_state_terminal) for each possible action outcome]}}


In [9]:
def compute_P_and_R(env, policy):
    # Initialize the matrices P and R
    n_states = env.observation_space.n
    P = np.zeros((n_states, n_states))

    R = np.empty((n_states, 1))
    R[:] = np.nan

    # Initialize a vector for terminal states
    # we will initialize with nan
    terminal_states = np.zeros(n_states)

    # Iterate over env.env.P.items()
    # env.env.P is a dictionary that maps from 
    # departure state to a dictionary of possible actions and subsequent transitions
    for state, action_dict in env.env.P.items():
        for action, transitions in action_dict.items():
            # Obtain the probability of taking that action
            # Given that there is equal probability for all possible actions
            # We set the probability of taking this action as 1/number of possible actions
            prop_action = 1/len(action_dict)

            # Iterate over all transitions for this action
            for transition in transitions:
                next_state = transition[1]
                probability = transition[0]
                reward = transition[2]

                # If the next state is terminal, flag it as such in terminal_states
                if transition[3] == True:
                    terminal_states[next_state] = 1

                # If we are not departing from a terminal state
                if terminal_states[state] == 0:
                    # Add to P the probability of the transition as:
                    # (probability of the action) * (probability of the transition)
                    P[state][next_state] += prop_action * probability

                    # Set the reward in R
                    R[next_state] = reward

    # Convert the terminal states vector to boolean to use in indexing
    terminal_states = terminal_states.astype(bool)

    return P, R, terminal_states

env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array", is_slippery = False))
P, R, terminal_states = compute_P_and_R(env, uniform_policy)

print(P)
print(R)
print(terminal_states)

[[0.5  0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.5  0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.   0.   0.25 0.25 0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.25 0.25 0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.25 0.   0.25 0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.

** Comments:**

The probaibilty matrix : We can get if from this formula , P(S = s' | S = s, A = a){probability of moving from the current state s to the next state s'and the current action taken is a}, and the rewrad matrix we find that just last values is 1 when he get the present , and terminal state matrix, we find that we have just 5 true when the game is finished (hole or present).



{state: {action: [(probability, next_state, reward, is_next_state_terminal) for each possible action outcome]}}

## Exercise : Sample an episode

Sample an episode with the uniform policy and return the lists of observations (states), actions, rewards and the done flags.

Notice that the list of states should have one more element (the initial state).

Test the function by rendering an episode. We will use `RecordVideo` to do that.

In [10]:
def sample_episode(env, policy, reset=True):
    states = []
    actions = []
    rewards = []
    dones = []

    ### BEGIN SOLUTION

    # If reset, we reset the environment and get an initial state
    # else we set the initial state to it's current state env.env.s

    if reset :
      state = env.reset()[0]
    else:
      state = env.env.s 
    # Collect the initial state

    states.append(state)
    done = False
    # While the episode has not finished
    while not done:

        # Select an action
        action = policy.sample(state)
        
        # Step the environment
        observation, reward, terminated, truncated, info = env.step(action)

        # The episode is done if it has been terminated or truncated
        done = terminated or truncated

        # Collect the state, reward and action taken
        states.append(observation)
        actions.append(action)
        rewards.append(reward)
        dones.append(done)

        state = observation 

    ### END SOLUTION

    return states, actions, rewards, dones

env = RecordVideo(gym.make("FrozenLake-v1", render_mode="rgb_array"))
uniform_policy = UniformPolicy(env.action_space)
states, actions, rewards, dones = sample_episode(env, uniform_policy)
print(rewards)
print(len(states), len(actions), len(rewards), len(dones))

display(env.video())

<IPython.core.display.Javascript object>

[0.0, 0.0, 0.0]
4 3 3 3


Comments:
here we eturn the lists of observations (states), actions, rewards and the done flags. for example here we find that, the all rewards is zero because the game don'st arrive to the present and there is a n actions, n rewards of zeros, and n false dones.

## Exercise : Returns of episode

For a sampled episode compute the return $G_t$ for all steps $t$.

**Hint** the return is easily computed backwards from the last step in the episode to the first.

In [11]:
def compute_returns(rewards, gamma):
    returns = np.zeros(len(rewards))

    ### BEGIN SOLUTION

    # Iterate over the rewards backward computing each return
    # using the previous return computed
    ### BEGIN SOLUTION
    G = 0
    for t in reversed(range(len(rewards))):
        G = rewards[t] + gamma*G
        returns[t] = G
    return returns


env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)

# Now we will run until one of them has a positive reward
while True:
    states, actions, rewards, dones = sample_episode(env, uniform_policy)

    if np.sum(rewards) > 0:
        # Print the rewards
        print(rewards)

        # Compute and print the returns
        gamma = 0.9
        returns = compute_returns(rewards, gamma)
        print(returns)

        # Exit the loop
        break

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.43046721 0.4782969  0.531441   0.59049    0.6561     0.729      0.81       0.9        1.        ]


This function return the return of spisode : this return is measure of the total future reward.

Gt = Rt + γRt+1 + γ^2Rt+2 + ... = ∑ γ^k Rt+k




# Policy evaluation

In this section we will implement various value function estimation methods:
- Direct method using the previous P and R
- Naive (cheating) method that assumes possibility of setting an arbitrary initial state
- Monte Carlo methods
- Time-difference method

## Exercise : Direct method

Solve the Bellman equation, using the direct solution (matrix inversion).

**Note** that this method leads to miss-leading values for terminal states, since there is no way to indicate terminality using the Bellman equation without sampling.

In [12]:
def value_direct(env, gamma, policy):
    P, R, terminal_states = compute_P_and_R(env, policy)
    ### BEGIN SOLUTION
    v=np.matmul(np.linalg.inv(np.identity(16)- gamma*P),R)

    ### END SOLUTION

    # Unsqueeze the extra dimension
    v = v[:,0]

    return v
    
env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
v_direct = value_direct(env, gamma, uniform_policy)
print(v_direct)

[0.00402953 0.00380021 0.00906008 0.0037064  0.00604976 0.         0.02370034 0.         0.01680854
 0.05184631 0.09627475 0.         0.         0.11734474 0.35234114 1.        ]


Solving the Bellaman equation: ひ = (I - γ P )^(-1) R

## (Optional) Exercise : Verify the solution

Check if the computed value function $V$ fulfills the Bellman equation. You may use `np.allclose(a, b)` to check if all elements in `a` and `b` are close up to a numerical error.

In [13]:
### BEGIN SOLUTION
a=((gamma*np.matmul(P,v_direct))+np.transpose(R))
np.allclose(v_direct, a)
### END SOLUTION

True

compare between ひ = R + γ P ひ and v_direct

For the following we must set the value function of terminal states to 0 in order to keep our value function comparable to those computed by methods using episode sampling.

In [14]:
# To compare to the following methods we must set the value of terminal states to 0
v_direct[terminal_states] = 0
print(v_direct)

[0.00402953 0.00380021 0.00906008 0.0037064  0.00604976 0.         0.02370034 0.         0.01680854
 0.05184631 0.09627475 0.         0.         0.11734474 0.35234114 0.        ]


## Exercise : Naive (cheating) method

Compute the value function for each state by cheating (changing the starting state and computing the average return from the start).

In [15]:
def value_naive(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION

    # Initialize values and counts tables (one cell per state)
    n_states = env.observation_space.n
    values = np.zeros(n_states)

    # Compute the number of episodes per state
    for initial_state in range(n_states):
        episode_return =0

        # For each episode
        for episode in range(n_episodes):

            # Reset the environment
            state = env.reset()

            # Set the initial state
            state = initial_state

            # Sample an episode 
            # (without reseting the environment to avoid changing the initial state)
            states, actions, rewards, dones= sample_episode(env,policy,reset=False)
            returns = compute_returns(rewards, gamma)
            # Accumulate the return of the initial state
            episode_return  += returns[0]

        # Divide the accumulated value by the number of episodes
        values[state] = episode_return/ n_episodes

    ### END SOLUTION
    
    return values

def compute_value_error(v_est, v_ref):
    diff = (v_est - v_ref)
    return np.mean(diff),np.std(diff)

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = int(10000/16)
v_naive = value_naive(env, gamma, uniform_policy, n_episodes)
print(v_naive)
print(compute_value_error(v_naive, v_direct))


[0.00333244 0.00331524 0.00480672 0.00496133 0.00300279 0.0041598  0.00307813 0.00397285 0.00191644
 0.00430899 0.00401638 0.0035105  0.00498717 0.00761086 0.00301643 0.00312316]
(-0.038865160509167176, 0.08711681468712161)


here we compare between the direct value and the cheating value, by changing the starting state and computing the average return from the start and we find that the differnce between them is small . So, i think it's good for estimation.

## Exercise : Monte Carlo first visit method
Compute the value function for each state using the Monte carlo method "first visit"

In [16]:
def value_montecarlo_first(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION

    # Initialize values and counts tables (one cell per state)
    n_states = env.observation_space.n
    values = np.zeros(n_states)
    counts = np.zeros(n_states)

    # For each episode
    for episode in range(n_episodes):

        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy)
        returns = compute_returns(rewards, gamma)
        # Keep track of visited states
        visited_states = set()
        # For each state and associated return
        for i in range (len(states)-1):

            # If first visit
            state = states[i]
            # if state not in visited_states:
            if state not in visited_states:

                # Increment counts
                counts[state] += 1

                # Accumulate returns
                values[state] += returns[i]

                # Update the set of visited states
                visited_states.add(state)

    # Average the accumulated return
    # value=values / counts
    value = np.divide(values, counts, out=np.zeros_like(counts), where=counts!=0 )

    ### END SOLUTION
    
    return value

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_firstvisit = value_montecarlo_first(env, gamma, uniform_policy, n_episodes)
print(v_mc_firstvisit)

print(compute_value_error(v_mc_firstvisit, v_direct))

[0.00421853 0.00434264 0.00951636 0.00413258 0.00654929 0.         0.02753951 0.         0.01878796
 0.05933689 0.10960858 0.         0.         0.13814346 0.39700161 0.        ]
(0.005888475035741205, 0.01152682650496435)


In montecarlo first visit, it offers good estimation for value function give a state. I think that the error will decrease if we sample more episodes.

## Exercise : Monte Carlo every visit method
Compute the value function for each state using the Monte carlo method "every visit"

In [17]:
def value_montecarlo_every(env, gamma, policy, n_episodes):
    n_states = env.observation_space.n
    values = np.zeros(n_states)
    counts = np.zeros(n_states)

    # For each episode
    for episode in range(n_episodes):

        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy)
        returns = compute_returns(rewards, gamma)
        
        # For each state and associated return
        for i in range (len(states)-1):

            # Increment counts
            counts[states[i]] += 1

            # Accumulate returns
            values[states[i]] += returns[i]

    # Average the accumulated returns
    value = np.divide(values, counts, out=np.zeros_like(counts), where=counts!=0 )

    ### END SOLUTION

    return value


env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_everyvisit = value_montecarlo_every(env, gamma, uniform_policy, n_episodes)
print(v_mc_everyvisit)
print(compute_value_error(v_mc_everyvisit, v_direct))


[0.00452226 0.00376659 0.01125216 0.00431597 0.0065094  0.         0.02796172 0.         0.01791347
 0.05321509 0.10215011 0.         0.         0.12608682 0.36161777 0.        ]
(0.0021468463806619178, 0.003063312448709212)


Every visit shows less error than first visit because it considers all visited states so i think it is more computationaly effecient but it might provide overstimation of the value because it might visit the same state many times.

## Exercise : Monte Carlo incremental

Implement the incremental Monte Carlo method.

In [18]:
def value_montecarlo_incremental(env, gamma, policy, n_episodes):
    ### BEGIN SOLUTION
    # Initialize values and counts tables (one cell per state)
    n_states = env.observation_space.n
    value = np.zeros(n_states)
    counts = np.zeros(n_states)

    # For each episode
    for e in range(n_episodes):

        # Sample an episode and compute returns
        states, actions, rewards, dones = sample_episode(env, policy)
        returns = compute_returns(rewards, gamma)

        # For each state and associated return
        for i in range (len(states[:-1])):
            retur =returns[i]

            # Increment counts
            counts[states[i]] += 1
            # Update value with return
            value[states[i]]+=(retur -value[states[i]])/counts[states[i]]

    ### END SOLUTION
    return value

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
v_mc_incremental = value_montecarlo_incremental(env, gamma, uniform_policy, n_episodes)
print(v_mc_incremental)

print(compute_value_error(v_mc_incremental, v_direct))

[0.00526333 0.00466777 0.00887216 0.00248889 0.0070711  0.         0.02532094 0.         0.01996696
 0.06583647 0.1159629  0.         0.         0.14713218 0.43676794 0.        ]
(0.00964930118090371, 0.021104230983987827)


It shown that the error decreased much when using incremental method. it offers better estimate. this method uses the incremental mean it using incremental mean might introduce some bais.

ひ(s ) = ひ(s ) + (Gt- ひ(s ))/N(s )

Here each time we visit a state for the first time, we make some add of the reward to the total rewards for that state and then we divide the total reward by the number of times we have visited. 


## Exercise : Time-differences (TD) method

In [19]:
def value_td(env, gamma, policy, n_episodes, alpha=0.4):
    ### BEGIN SOLUTION
    # Initialize value table (one cell per state)
    n_states = env.observation_space.n
    value = np.zeros(n_states)
    counts = np.zeros(n_states)


    # For each episode
    for e in range(n_episodes):

        # Sample an episode
        states, actions, rewards, dones = sample_episode(env, policy)
        returns = compute_returns(rewards, gamma)

        # For each step in the episode
        for i in range (len(states)-1):
            rewa =rewards[i]

            # Increment counts
            counts[states[i]] += 1

            # Update the value of the depart state with the current value, the value of the next state and the reward
            value[states[i]]=value[states[i]] + alpha * (rewa+ gamma * value[states[i+1]] - value[states[i]] )


    ### END SOLUTION
    return value

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
alpha = 0.4
v_td = value_td(env, gamma, uniform_policy, n_episodes, alpha=alpha)
print(v_td)
print(compute_value_error(v_td, v_direct))

[9.17540540e-04 2.92926519e-04 2.01131330e-03 1.33343749e-04 1.82207042e-03 0.00000000e+00
 2.92491612e-03 0.00000000e+00 5.90927136e-03 5.85713083e-02 3.77718559e-02 0.00000000e+00
 0.00000000e+00 1.19626342e-01 6.27934071e-01 0.00000000e+00]
(0.010809571967042884, 0.0699140790194549)


I think that using TD method offers stability low variance but i can't see that in this example. I think it will work in other environment but in this case using montecarlo methods is better

A simple TD algorithm is :

ひ(st) = ひ(st) + α(Rt+ γひ(st+1 ) - ひ(st))

## (Optional) Exercise : Comparison TD to Monte Carlo incremental

How do these to methods compare? Explain the relative advantages and disadvantages.

Two different methouds to update the value functions, the differnce between them ( how they update the value function): 

**Temporal Difference vs Monte Carlo**

Monte Carlo

Estimation of ひπ(s) is unbiased, meaning it gives an accurate representation of the true value of a state.

Variance is large, meaning it requires a large number of samples (i.e. a lot of random choices in an episode) to get a good estimate of the value of a state.

Not sensitive to the initial condition, meaning it doesn't matter where you start the estimation process.

Temporal Difference (TD)

Estimation of ひπ(s) is biased, meaning it might not give an accurate representation of the true value of a state.

Variance is low, meaning it requires fewer samples to get a good estimate of the value of a state.

Can learn in the middle of an episode, meaning it can start adjusting the values of states even before the episode ends.

Can learn from incomplete sequences, meaning it doesn't need a full episode to start making updates.


## Exercise : Action-value
Compute the action value function using one or more of the following methods:
- Naive (cheating)
- MC "first visit"
- MC "every visit"
- MC incremental
- TD

In [20]:
#applyning MC "every visit"
def actionvalue_montecarlo_every(env, gamma, policy, n_episodes):
      ### BEGIN SOLUTION
    # Initialize the action-value function and the counts table (one cell per state-action pair)
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    q = np.zeros((n_states, n_actions))
    counts = np.zeros((n_states, n_actions))

    # For each episode
    for e in range(n_episodes):

        # Sample an episode
        states, actions, rewards, dones = sample_episode(env, policy)
        returns = compute_returns(rewards, gamma)

        # For each step in the episode
        for i in range(len(states)-1):
            state = states[i]
            action = actions[i]
            ret = returns[i]

            # Increment counts
            counts[state][action] += 1

            # Update the value of the state-action
            q[state][action] += ret
    q = np.divide(q, counts, out=np.zeros_like(counts), where=counts!=0 )

    # Return the computed action-value function
        ### END SOLUTION
    return q
    
env = gym.make("FrozenLake-v1", render_mode="rgb_array")
uniform_policy = UniformPolicy(env.action_space)
gamma = 0.9
n_episodes = 10000
q_mc_everyvisit = actionvalue_montecarlo_every(env, gamma, uniform_policy, n_episodes)
print(q_mc_everyvisit)

[[0.00552276 0.00463423 0.00403066 0.00423549]
 [0.00159209 0.00300682 0.00429537 0.00626315]
 [0.00786238 0.00873688 0.01113039 0.00304282]
 [0.00256096 0.00330055 0.00077583 0.00505454]
 [0.00962886 0.00835006 0.00788181 0.00441566]
 [0.         0.         0.         0.        ]
 [0.02781637 0.02717757 0.02345037 0.00225762]
 [0.         0.         0.         0.        ]
 [0.00881899 0.024223   0.01828791 0.02691732]
 [0.05311647 0.08514566 0.05516477 0.03540992]
 [0.08899894 0.12330961 0.12466246 0.0158077 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.05782661 0.15610328 0.21840618 0.13619536]
 [0.21165269 0.42782227 0.46482227 0.38169641]
 [0.         0.         0.         0.        ]]


# Policy improvement

In this section we are going to improve upon the uniform policy, which selects actions at random, independently on the state.

To assess whether our learned policies work, we will start by implmenting a scoring function to evaluate the policies.

## Exercise : Evaluate performance of a policy

In this environment we define a goal as obtaining a final reward greater than 0, thus reaching the target state, since it is the only one with a non-zero reward.

For a given policy, compute:
- the average number of episodes where the goal was reached
- the average number of steps to reach the goal


In [21]:
def score_policy(env, gamma, policy, n_episodes):
    episodes_to_goal = 0
    steps_to_goal = []

    ### BEGIN SOLUTION
    # Counter 
    counter = 0
    # Loop through each episode
    for ep in range(n_episodes):
        
        # Sample the episode
        states, actions, rewards, dones = sample_episode(env, policy)
        # Calculate the sum of rewards 
        reward_sum = np.sum(rewards)
        # Check if the sum of rewards is greater than 0
        if reward_sum > 0:
            # If the sum of rewards is greater than 0, increment the episode to goal counter
            episodes_to_goal += 1
            # Add the length of actions to the steps to goal list
            steps_to_goal.append(len(actions))
        # Add the sum of rewards for this episode to the counter
        counter += reward_sum
    ### END SOLUTION

    return episodes_to_goal/n_episodes, np.mean(steps_to_goal)

## Exercise : Greedy policy (policy improvement)

Create a greedy policy from an action value function and evaluate it's performance.

The greedy policy selects the action leading to the largest value in the current state.

**Note** that when several actions have the same maximal value, it is best to randomly pick one of them.

Pick the first highest value action or (optionally) pick randomly amongst actions with the highest value.

In [22]:
class GreedyPolicy(UniformPolicy):
    def __init__(self, action_space, q):
        super().__init__(action_space)

        # q is the action-value table
        self.q = q

    def _max_value_action(self, state):
        ### BEGIN SOLUTION

        # action = np.argmax(self.q[state])
        # Find the action with the highest value for the given state
        action = np.argmax(self.q[state])

        ### END SOLUTION
        return action

    def probability(self, state, action):
        # Select the highest value action
        action_max = self._max_value_action(state)

        # Return a probability of 1 for the selected action 0 otherwise
        action_prob = float(action == action_max)
        
        return action_prob

    def sample(self, state):
        # Select the highest value action
        action_max = self._max_value_action(state)

        return action_max

env = gym.make("FrozenLake-v1", render_mode=None)
greedy_policy = GreedyPolicy(env.action_space, q_mc_everyvisit)

## Exercise : Compare the performance of policies

Report the performance of the uniform policy and the greedy policy

In [23]:
env = gym.make("FrozenLake-v1", render_mode=None)

### BEGIN SOLUTION
uniform_policy_score = score_policy(env, gamma, uniform_policy, n_episodes)
greedy_policy_score = score_policy(env, gamma, greedy_policy, n_episodes)

print (f'uniform policy: {uniform_policy_score[0]:.1%} / {uniform_policy_score[1]}')
print (f'greedy policy: {greedy_policy_score[0]:.1%} / {greedy_policy_score[1]}')

### END SOLUTION

uniform policy: 1.3% / 13.03125
greedy policy: 36.9% / 29.513147194361615


uniform policy score is less that 2% because it selects random action every time so it is not a good policy but greedy policy is good policy for this environment but it needs improvement but introducing exploring term.

# Policy learning

In this section we will start learning policies.

We will then implement the following policy learning methods:
- policy iteration
- SARSA
- Q-Learning

## Exercise : Policy iteration
By alternating between policy evaluation and policy improvement, find an optimal policy.

Print the scores of each intermediate policy and comment on how the metrics evolve.

In [24]:
def policy_learn_iteration(env, initial_policy, policy_evaluation_function, 
                           n_episodes_value, n_episodes_score, n_iterations):
    policy = initial_policy

    ### BEGIN SOLUTION
    # Score and print the initial policy
    score = score_policy(env, gamma, policy, n_episodes_score)
    print (f'initial policy {score[0]:.1%} / {score[1]}')


    for i in range(n_iterations):
        # Policy evaluation
        q = policy_evaluation_function(env,gamma, policy, n_episodes_value)


        # Keep track of the action-value function change 
        # (sum of absolute difference between the previous 
        #  and next action-value funcitons)
        # Note that at the first step we will report a change of nan
        # since we still don't have a previous action-value function
        q_change = np.abs(q - q_old).sum() if i > 0 else np.nan
        q_old = q

        # Policy improvement
        policy = GreedyPolicy(env.action_space, q) 

        # Policy scoring
        score = score_policy(env, gamma, policy, n_episodes_score)

        print(f'Policy (iter: {i}): {score[0]:.1%} / {score[1]}, Value change: {q_change}')


    ### END SOLUTION

env = gym.make("FrozenLake-v1", render_mode=None)
uniform_policy = UniformPolicy(env.action_space)

policy_learn_iteration(env, uniform_policy, actionvalue_montecarlo_every, 
                       n_episodes_value=1000, n_episodes_score=10000,
                       n_iterations=100)

initial policy 1.4% / 12.583333333333334
Policy (iter: 0): 9.3% / 13.237068965517242, Value change: nan
Policy (iter: 1): 8.8% / 13.566742081447964, Value change: 3.05692636041179
Policy (iter: 2): 8.5% / 13.208920187793428, Value change: 0.23835416623982972
Policy (iter: 3): 9.1% / 13.655286343612335, Value change: 0.13896785675568485
Policy (iter: 4): 9.2% / 13.371212121212121, Value change: 0.09542732378779599
Policy (iter: 5): 9.7% / 13.329545454545455, Value change: 0.3255138144243783
Policy (iter: 6): 9.3% / 13.29784946236559, Value change: 0.3048045948373366
Policy (iter: 7): 9.4% / 13.186965811965813, Value change: 0.16901445117733122
Policy (iter: 8): 9.0% / 13.714922048997773, Value change: 0.27542084153725027
Policy (iter: 9): 8.8% / 13.34738041002278, Value change: 0.1302668461052742
Policy (iter: 10): 9.4% / 13.48936170212766, Value change: 0.2703382627678098
Policy (iter: 11): 8.5% / 13.301997649823736, Value change: 0.46933168657968993
Policy (iter: 12): 9.4% / 13.223516

The reault is fluctuating and not improving, We see that the percentage of scores is around 9%, and I think this is a low success rate. 

## Exercise : Epsilon-greedy policy

One of the issues with the greedy policy is that some possible trajectories may never be visited depending on the initial estimation of the value function.

To avoid this create an epsilon-greedy policy. These policies act differently when running in training mode and evaluation mode. In evaluation mode they act as a greedy policy. In training mode:
- with probability epsilon, uniformly selects an action
- else selects the action with maximum value (greedy policy)

Implement the `EpsilonGreedyPolicy` class.


In [25]:
class EpsilonGreedyPolicy(GreedyPolicy):
    def __init__(self, action_space, q, epsilon, 
                 epsilon_decay=1, epsilon_min=0):
        super().__init__(action_space, q)
        self.epsilon_start = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        self.epsilon = self.epsilon_start

    def sample(self, state):
      ### BEGIN SOLUTION
        if np.random.rand() < self.epsilon:
            #uniformly select an action
            action = np.random.choice([0, 1, 2, 3])
        else:
            # Select the action with maximum value (greedy policy)
            action = self._max_value_action(state)
        return action

        # ### END SOLUTION

    def begin_episode(self, episode_index):
        # Start of an episode
        self.epsilon = self.epsilon_start * (self.epsilon_decay ** episode_index)
        self.epsilon = max(self.epsilon, self.epsilon_min)

# Instantiate a policy
dummy = EpsilonGreedyPolicy(env.action_space, q_mc_everyvisit, epsilon=0.5)

# Sample 20 actions from state 0 in train mode
dummy.train()
actions = [dummy.sample(0) for i in range(20)]
print(actions)

# Sample 20 actions from state 0 in eval mode
dummy.eval()
actions = [dummy.sample(0) for i in range(20)]
print(actions)

[0, 0, 0, 0, 3, 0, 0, 3, 1, 1, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0]
[0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0]


## Exercise : SARSA

State–action–reward–state–action (SARSA) is a method to learn a policy leveraging the TD estimation of action-value functions and an epsilon-greedy policy.

Since it is based on TD, it does not require full episodes to train, the policy can be improved at each step. We will therefore **not use the `sample_episode`** and reimplement here a similar loop.

Implement the SARSA algorithm, in this case it will be passed a policy of type `EpsilonGreedyPolicy` which stores the action-value (or Q-table). We may access and modify if using `policy.q`.

Score the trained policy.

In [26]:
def policy_learn_sarsa(env, policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500):
    # Loop through each episode
    for episode in range(n_episodes):
        
        # Call the policy's begin_episode method so it can handle epsilon decay
        policy.begin_episode(episode)
        
        # Print the episode and epsilon values every `print_every` episodes
        if not episode % print_every:
            print(f'ep: {episode}, epsilon: {policy.epsilon:.3f}')

        # Get the initial state and information from the environment
        state, info = env.reset()

        # Sample an action using the policy
        action = policy.sample(state)

        # Loop through the steps in the episode
        for step in range(max_n_steps):
            # Perform a step in the environment 
            state_next, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            # If episode has finished
            if done :
                # Update the action-value table and leave the loop
                policy.q[state, action] += alpha * (reward - policy.q[state, action])
                break

            # Sample the next action
            action_next = policy.sample(state_next)

            # Update the action-value table
            policy.q[state, action] += alpha * (reward + gamma * policy.q[state_next, action_next] - policy.q[state, action])
            state = state_next
            action = action_next
    
    return
env = gym.make("FrozenLake-v1", render_mode=None)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
sarsa_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                   epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 10000
alpha = 0.2
n_episodes_score = 10000

### BEGIN SOLUTION
# Train the policy
sarsa_policy.train()
policy_learn_sarsa (env, sarsa_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

# Evaluate the policy

sarsa_policy.eval()
avg_episodes_to_goal, avg_steps_to_goal = score_policy(env, gamma, sarsa_policy, n_episodes_score)
print(f'Policy SARSA: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')

ep: 0, epsilon: 1.000
ep: 500, epsilon: 0.606
ep: 1000, epsilon: 0.368
ep: 1500, epsilon: 0.223
ep: 2000, epsilon: 0.135
ep: 2500, epsilon: 0.082
ep: 3000, epsilon: 0.050
ep: 3500, epsilon: 0.030
ep: 4000, epsilon: 0.018
ep: 4500, epsilon: 0.011
ep: 5000, epsilon: 0.010
ep: 5500, epsilon: 0.010
ep: 6000, epsilon: 0.010
ep: 6500, epsilon: 0.010
ep: 7000, epsilon: 0.010
ep: 7500, epsilon: 0.010
ep: 8000, epsilon: 0.010
ep: 8500, epsilon: 0.010
ep: 9000, epsilon: 0.010
ep: 9500, epsilon: 0.010
Policy SARSA: 42.53% / 27.89


The percentage of plicy for SARSA is 42.53% and this is low success rate, and maybe differ with other environemnt, and here we observe that the score depends on the decay rate.

## Exercise : Q-learning

Q-learning is very similar to SARSA. **SARSA is an on-policy learning method** in which the action-values are updated following the same policy. In SARSA the action-values are updated using the value of the next state and next action taken.

Q-learning is off-policy, it does not assume the same policy when updating the action-values, instead it assumes an optimal policy by using the maximum value of the next state.

Implement a modified version of `policy_learn_sarsa` that performs Q-learning.

In [27]:
def policy_learn_qlearn(env, policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500):
    # No need to create the tables of action-values since they are stored directly in the policy
    
    # While we haven't reached the desired number of steps
    for episode in range(n_episodes):
        # Call the policy begin_episode so it can handle epsilon decay
        policy.begin_episode(episode)
        
        # Print every couple episodes
        if not episode % print_every:
            print(f'ep: {episode}, epsilon: {policy.epsilon:.3f}')

        # Get initial state
        state, info = env.reset()

        # Take the first action acording to the policy
        action = policy.sample(state)

        # While we haven't reached the maximum number of steps for the episode
        for step in range(max_n_steps):     
            ### BEGIN SOLUTION
            # Perform a step of the environment
            state_next, reward, terminated, truncated, info = env.step(action)


            # Set as done if terminated or truncated
            done = terminated or truncated

            # If episode has finished
            if done or step == max_n_steps-1:
                # Update the action-value table and leave the loop
                policy.q[state][action] += alpha * (reward - policy.q[state][action])


            # Sample the next action
            action_next = policy.sample(state_next)

            # Update the action-value table
            policy.q[state][action] += alpha * (reward + gamma * np.max(policy.q[state_next]) - policy.q[state][action]) 

            ### END SOLUTION

            # Set the current state and action
            state = state_next
            action = action_next
            
    return

env = gym.make("FrozenLake-v1", render_mode=None, is_slippery = False)

q_initial = np.zeros((env.observation_space.n, env.action_space.n))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 1000

### BEGIN SOLUTION
# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)

# Evaluate the policy
qlearn_policy.eval()
avg_episodes_to_goal, avg_steps_to_goal = score_policy(env,gamma, qlearn_policy, n_episodes_score)

print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')


ep: 0, epsilon: 1.000
ep: 500, epsilon: 0.606
ep: 1000, epsilon: 0.368
ep: 1500, epsilon: 0.223
ep: 2000, epsilon: 0.135
ep: 2500, epsilon: 0.082
ep: 3000, epsilon: 0.050
ep: 3500, epsilon: 0.030
ep: 4000, epsilon: 0.018
ep: 4500, epsilon: 0.011
Policy Q-learn: 98.70% / 6.04


here we find that the q learning is the best methoud, because has a high percentage of policy about "98.70%", which is a very high success rate. And this indicating a good performance. and Q-learningis better in the deterministic environments.

## (Optional) Exercise : Train for Taxi

Train an epsilon-greedy policy using Q-learning on the `Taxi-v3` environment.

Score the performance of this policy and compare it to a uniform policy.

In [28]:
### BEGIN SOLUTION
#intialize
env = gym.make("Taxi-v3", render_mode=None)
#intialize policy
q_initial = np.zeros((env.observation_space.n, env.action_space.n))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 10000

# Train and evaluate the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)
qlearn_policy.eval()


avg_episodes_to_goal, avg_steps_to_goal = score_policy(env,gamma, qlearn_policy, n_episodes_score)

print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')


ep: 0, epsilon: 1.000
ep: 500, epsilon: 0.606
ep: 1000, epsilon: 0.368
ep: 1500, epsilon: 0.223
ep: 2000, epsilon: 0.135
ep: 2500, epsilon: 0.082
ep: 3000, epsilon: 0.050
ep: 3500, epsilon: 0.030
ep: 4000, epsilon: 0.018
ep: 4500, epsilon: 0.011
Policy Q-learn: 99.90% / 13.22


here we find that because Taxi is a deterministic environment, so it has an excellent poilcy here.

## (Optional) Exercise : Render Taxi

Run 3 episodes of a Q-learn trained policy on `Taxi-v3` this time rendering the result.

In [29]:
### BEGIN SOLUTION

#Initialize the environment
env = RecordVideo(gym.make("Taxi-v3", render_mode='rgb_array'))
env.reset()
qlearn_policy.eval()
for i in range(3):
  _,_,_,_ = sample_episode(env, qlearn_policy)
env.close()


### END SOLUTION

<IPython.core.display.Javascript object>

## (Bonus) Exercise : A observation space environment

Try to perform Q-learning on an environment (e.g. `CartPole-v1`) with continuous action and observation spaces.

You will need to discretize the observation space.

In [30]:
# Example of rendering and showing a CartPole-v1 environment
env = RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"))
print(env.action_space)
print(env.observation_space)

observation, info = env.reset()

while True:
    env.render()
    
    action = env.action_space.sample() 
         
    state_next, reward, terminated, truncated, info = env.step(action)

    done = terminated or truncated
   
    if done: 
      break;
            
env.close()

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


<IPython.core.display.Javascript object>

In [78]:
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_states=4096, bounds=(-50, 50)):
        super().__init__(env)

        self.n_states = n_states

        assert isinstance(env.observation_space, gym.spaces.box.Box)

        ### BEGIN SOLUTION
        # Get the dimensions of the observation space
        self.dim = env.observation_space.shape

        # Decide how many bins per dimension based on the target n_states
        self.n_bins = int(np.sqrt(self.n_states/self.dim[0]))

        # Define the bins to discretize based on the bounds and self.n_bins
        self.bins = np.linspace(bounds[0], bounds[1], self.n_bins + 1)[1:-1]

        ### END SOLUTION
        
    def observation(self, obs):
        ### BEGIN SOLUTION
        # Quantize the observations (states)
        obs = np.digitize(obs, bins=self.bins)

        # Once quantized each dimension compute a single observation index
        new_obs = np.ravel_multi_index(obs, [self.n_bins] * self.dim[0])

        ### END SOLUTION
        return new_obs

# Initialize environment, wrap to render
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Wrap to discretize the observation space
env = ObservationWrapper(env)

### BEGIN SOLUTION
#intialize policy
q_initial = np.zeros((env.n_bins ** env.dim[0], env.action_space.n))
qlearn_policy = EpsilonGreedyPolicy(env.action_space, q_initial,
                                    epsilon=1, epsilon_decay=0.999, epsilon_min=0.01)

gamma = 0.9
n_episodes = 5000
alpha = 0.2
n_episodes_score = 10000

# Train the policy
qlearn_policy.train()
policy_learn_qlearn(env, qlearn_policy, gamma, n_episodes, alpha, max_n_steps=1000, print_every=500)


avg_episodes_to_goal, avg_steps_to_goal = score_policy(env,gamma, qlearn_policy, n_episodes_score)

print(f'Policy Q-learn: {avg_episodes_to_goal:.2%} / {avg_steps_to_goal:.2f}')

# Recreate the wrapped environment 
env = gym.make("CartPole-v1", render_mode="rgb_array")
env = ObservationWrapper(env)


# Render and show an episode with the trained policy
env = RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"))

env.reset()
qlearn_policy.eval()
for i in range(3):
  _,_,_,_ = sample_episode(env, qlearn_policy)
env.close()
### END SOLUTION

ep: 0, epsilon: 1.000


AssertionError: ignored

I just follwoing the comments, and i don't know what's the error here but i think the error in class EpsilonGreedyPolicy(GreedyPolicy):
beecause the class has actions of 4 action and it's suitable for the FrozenLake-v1, and CartPole-v1 consists of two actions: moving left or moving right.so maybe we have to edit the greedypolice